<a href="https://colab.research.google.com/github/shafaqueahmareen/Kmeans_Pyspark/blob/main/kmeans_session1_seed_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425345 sha256=6ce205c54d4adeb7a195e570bda7b5f4ae4169b4d1f3673211ebd82b213dbbfd
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [ ]:
from pyspark.sql import SparkSession
from pyspark.ml.clustering import KMeans
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import StandardScaler

# create the spark session
spark  = SparkSession.builder.appName('KmeansExample').getOrCreate()

# read the dataset
path = '/content/seeds_dataset.csv'

data = spark.read.csv(path,header=True,inferSchema=True)
data.show()



+-----+---------+-----------+------------------+------------------+---------------------+------------------+
| area|perimeter|compactness|  length_of_kernel|   width_of_kernel|asymmetry_coefficient|  length_of_groove|
+-----+---------+-----------+------------------+------------------+---------------------+------------------+
|15.26|    14.84|      0.871|             5.763|             3.312|                2.221|              5.22|
|14.88|    14.57|     0.8811| 5.553999999999999|             3.333|                1.018|             4.956|
|14.29|    14.09|      0.905|             5.291|3.3369999999999997|                2.699|             4.825|
|13.84|    13.94|     0.8955|             5.324|3.3789999999999996|                2.259|             4.805|
|16.14|    14.99|     0.9034|5.6579999999999995|             3.562|                1.355|             5.175|
|14.38|    14.21|     0.8951|             5.386|             3.312|   2.4619999999999997|             4.956|
|14.69|    14.49|  

In [ ]:
data.columns

['area',
 'perimeter',
 'compactness',
 'length_of_kernel',
 'width_of_kernel',
 'asymmetry_coefficient',
 'length_of_groove']

In [ ]:
# convertiing the columns into feature to process in Kmeans
vec_assemble = VectorAssembler(inputCols=data.columns,outputCol='Features')
final_data = vec_assemble.transform(data)
final_data.show()

+-----+---------+-----------+------------------+------------------+---------------------+------------------+--------------------+
| area|perimeter|compactness|  length_of_kernel|   width_of_kernel|asymmetry_coefficient|  length_of_groove|            Features|
+-----+---------+-----------+------------------+------------------+---------------------+------------------+--------------------+
|15.26|    14.84|      0.871|             5.763|             3.312|                2.221|              5.22|[15.26,14.84,0.87...|
|14.88|    14.57|     0.8811| 5.553999999999999|             3.333|                1.018|             4.956|[14.88,14.57,0.88...|
|14.29|    14.09|      0.905|             5.291|3.3369999999999997|                2.699|             4.825|[14.29,14.09,0.90...|
|13.84|    13.94|     0.8955|             5.324|3.3789999999999996|                2.259|             4.805|[13.84,13.94,0.89...|
|16.14|    14.99|     0.9034|5.6579999999999995|             3.562|                1.355| 

In [ ]:
#Scaling of the data

scData = StandardScaler(inputCol='Features',outputCol='scaledFeatures',
                        withStd=True,withMean=False)

final_data = scData.fit(final_data).transform(final_data)
final_data.show()

+-----+---------+-----------+------------------+------------------+---------------------+------------------+--------------------+--------------------+
| area|perimeter|compactness|  length_of_kernel|   width_of_kernel|asymmetry_coefficient|  length_of_groove|            Features|      scaledFeatures|
+-----+---------+-----------+------------------+------------------+---------------------+------------------+--------------------+--------------------+
|15.26|    14.84|      0.871|             5.763|             3.312|                2.221|              5.22|[15.26,14.84,0.87...|[5.24452795332028...|
|14.88|    14.57|     0.8811| 5.553999999999999|             3.333|                1.018|             4.956|[14.88,14.57,0.88...|[5.11393027165175...|
|14.29|    14.09|      0.905|             5.291|3.3369999999999997|                2.699|             4.825|[14.29,14.09,0.90...|[4.91116018695588...|
|13.84|    13.94|     0.8955|             5.324|3.3789999999999996|                2.259|     

In [ ]:
#Train the K-means model

kmModel = KMeans(featuresCol='scaledFeatures',k=4,predictionCol='predLabel')
kmModelTr = kmModel.fit(final_data)




<bound method KMeansModel.clusterCenters of KMeansModel: uid=KMeans_4844224df85b, k=4, distanceMeasure=euclidean, numFeatures=7>

In [ ]:
output = kmModelTr.transform(final_data)
output.select('predLabel').show()

+---------+
|predLabel|
+---------+
|        1|
|        1|
|        1|
|        1|
|        1|
|        1|
|        1|
|        1|
|        0|
|        1|
|        1|
|        1|
|        1|
|        1|
|        1|
|        1|
|        2|
|        1|
|        1|
|        2|
+---------+
only showing top 20 rows



In [ ]:
from pyspark.ml.evaluation import ClusteringEvaluator

shEval = ClusteringEvaluator(featuresCol = 'scaledFeatures',predictionCol='predLabel')

ssd = shEval.evaluate(output) # transform the final output of the kmeans
print(ssd)

0.5148598460324953


In [ ]:
cl=[2,3,4,5,6,7,8,9,10]
for i in range(len(cl)):
  kmModel = KMeans(featuresCol='scaledFeatures',k=cl[i],predictionCol='predLabel')
  kmModelTr = kmModel.fit(final_data)
  output = kmModelTr.transform(final_data)
  shEval = ClusteringEvaluator(featuresCol = 'scaledFeatures',predictionCol='predLabel')
  ssd = shEval.evaluate(output) # transform the final output of the kmeans
  print(ssd,' ',end=' ')

0.6613125393827092   0.5928460631863854   0.5148598460324953   0.4567109949709705   0.45404640201475277   0.4349644654428388   0.3390901987865649   0.3975305231411343   0.36370972086120756   

In [ ]:
# train the model (kmeans) on the Mall customer dataset


path = '/content/Mall_Customers.csv'

data = spark.read.csv(path,header=True,inferSchema=True)
data.show()

+----------+------+---+------------+-------------+
|CustomerID|Gender|Age|AnnualIncome|SpendingScore|
+----------+------+---+------------+-------------+
|         1|  Male| 19|          15|           39|
|         2|  Male| 21|          15|           81|
|         3|Female| 20|          16|            6|
|         4|Female| 23|          16|           77|
|         5|Female| 31|          17|           40|
|         6|Female| 22|          17|           76|
|         7|Female| 35|          18|            6|
|         8|Female| 23|          18|           94|
|         9|  Male| 64|          19|            3|
|        10|Female| 30|          19|           72|
|        11|  Male| 67|          19|           14|
|        12|Female| 35|          19|           99|
|        13|Female| 58|          20|           15|
|        14|Female| 24|          20|           77|
|        15|  Male| 37|          20|           13|
|        16|  Male| 22|          20|           79|
|        17|Female| 35|        

In [ ]:
vec_assemble1 = VectorAssembler(inputCols=['AnnualIncome','SpendingScore'],outputCol='Features')
final_data1 = vec_assemble1.transform(data)
final_data1.show()

+----------+------+---+------------+-------------+-----------+
|CustomerID|Gender|Age|AnnualIncome|SpendingScore|   Features|
+----------+------+---+------------+-------------+-----------+
|         1|  Male| 19|          15|           39|[15.0,39.0]|
|         2|  Male| 21|          15|           81|[15.0,81.0]|
|         3|Female| 20|          16|            6| [16.0,6.0]|
|         4|Female| 23|          16|           77|[16.0,77.0]|
|         5|Female| 31|          17|           40|[17.0,40.0]|
|         6|Female| 22|          17|           76|[17.0,76.0]|
|         7|Female| 35|          18|            6| [18.0,6.0]|
|         8|Female| 23|          18|           94|[18.0,94.0]|
|         9|  Male| 64|          19|            3| [19.0,3.0]|
|        10|Female| 30|          19|           72|[19.0,72.0]|
|        11|  Male| 67|          19|           14|[19.0,14.0]|
|        12|Female| 35|          19|           99|[19.0,99.0]|
|        13|Female| 58|          20|           15|[20.0

In [ ]:
scData = StandardScaler(inputCol='Features',outputCol='scaledFeatures',
                        withStd=True,withMean=False)

final_data1 = scData.fit(final_data1).transform(final_data1)
final_data1.show()

+----------+------+---+------------+-------------+-----------+--------------------+
|CustomerID|Gender|Age|AnnualIncome|SpendingScore|   Features|      scaledFeatures|
+----------+------+---+------------+-------------+-----------+--------------------+
|         1|  Male| 19|          15|           39|[15.0,39.0]|[0.57110829030364...|
|         2|  Male| 21|          15|           81|[15.0,81.0]|[0.57110829030364...|
|         3|Female| 20|          16|            6| [16.0,6.0]|[0.60918217632388...|
|         4|Female| 23|          16|           77|[16.0,77.0]|[0.60918217632388...|
|         5|Female| 31|          17|           40|[17.0,40.0]|[0.64725606234413...|
|         6|Female| 22|          17|           76|[17.0,76.0]|[0.64725606234413...|
|         7|Female| 35|          18|            6| [18.0,6.0]|[0.68532994836437...|
|         8|Female| 23|          18|           94|[18.0,94.0]|[0.68532994836437...|
|         9|  Male| 64|          19|            3| [19.0,3.0]|[0.72340383438

In [ ]:
#Train the K-means model

kmModel = KMeans(featuresCol='scaledFeatures',k=4,predictionCol='predLabel')
kmModelTr = kmModel.fit(final_data1)

output = kmModelTr.transform(final_data1)
output.select('predLabel').show()

+---------+
|predLabel|
+---------+
|        3|
|        1|
|        3|
|        1|
|        3|
|        1|
|        3|
|        1|
|        3|
|        1|
|        3|
|        1|
|        3|
|        1|
|        3|
|        1|
|        3|
|        1|
|        3|
|        1|
+---------+
only showing top 20 rows



In [ ]:
shEval = ClusteringEvaluator(featuresCol = 'scaledFeatures',predictionCol='predLabel')

ssd = shEval.evaluate(output) # transform the final output of the kmeans
print(ssd)

0.5314957478395654


In [ ]:
cl=[2,3,4,5,6,7,8,9,10]
for i in range(len(cl)):
  kmModel = KMeans(featuresCol='scaledFeatures',k=cl[i],predictionCol='predLabel')
  kmModelTr = kmModel.fit(final_data1)
  output = kmModelTr.transform(final_data1)
  ClusteringEvaluator()
  shEval = ClusteringEvaluator(featuresCol = 'scaledFeatures',predictionCol='predLabel')
  ssd = shEval.evaluate(output) # transform the final output of the kmeans
  print(ssd)